<a href="https://colab.research.google.com/github/AbhiDhariwal/Hindi-NER/blob/master/Hindi_Spacy_Working_NER_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1




###**Kindly upload the train and val file**

Steps invloved :-

 1. loading hindi vectors from fasttext
 2. Loading pickle file of Traning and Testing Data



In [6]:
!wget https://raw.githubusercontent.com/AbhiDhariwal/Hindi-NER/master/CoNLL2Spacy.py

--2020-03-24 19:09:59--  https://raw.githubusercontent.com/AbhiDhariwal/Hindi-NER/master/CoNLL2Spacy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1128 (1.1K) [text/plain]
Saving to: ‘CoNLL2Spacy.py’

CoNLL2Spacy.py      100%[===================>]   1.10K  --.-KB/s    in 0s      

2020-03-24 19:09:59 (233 MB/s) - ‘CoNLL2Spacy.py’ saved [1128/1128]



In [0]:
from CoNLL2Spacy import *

In [11]:
file = open("val.txt", "r",encoding = "utf-8") 
valList = []
for line in file:
    valList.append(line[:-1])
valList[:15]

['आज U-date',
 'मौसम O',
 'कैसा O',
 'है O',
 '',
 'क्या O',
 'हरिद्वार U-location',
 'में O',
 'बहुत O',
 'ठंड U-weather_type',
 'है O',
 'अभी U-date',
 '',
 'आज U-date',
 'इतनी O']

In [0]:
TEST_DATA = conll2spacy(valList)

In [13]:
file = open("train.txt", "r",encoding = "utf-8") 
trainList = []
for line in file:
    trainList.append(line[:-1])
trainList[:15]

['अरे O',
 'सारथी O',
 'जम्मू U-location',
 'कैसा O',
 'मौसम O',
 'है O',
 '',
 'अभी U-date',
 'दिल्ली U-location',
 'में O',
 'कैसा O',
 'मौसम O',
 'है O',
 '',
 'आज U-date']

In [0]:
TRAIN_DATA = conll2spacy(trainList)

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
# downloading fasttext hindi vector
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.vec

--2020-03-24 19:11:42--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 416644179 (397M) [binary/octet-stream]
Saving to: ‘wiki.hi.vec’

wiki.hi.vec         100%[===================>] 397.34M  40.0MB/s    in 10s     

2020-03-24 19:11:53 (38.4 MB/s) - ‘wiki.hi.vec’ saved [416644179/416644179]



In [0]:
import spacy
import numpy
import pickle


In [0]:
lang = "hi"
vectors_loc = "wiki.hi.vec"
nlp = spacy.blank(lang)    
with open(vectors_loc, "rb") as file_:        
    header = file_.readline()        
    nr_row, nr_dim = header.split()        
    nlp.vocab.reset_vectors(width=int(nr_dim))        
    for line in file_:            
        line = line.rstrip().decode("utf8")            
        pieces = line.rsplit(" ", int(nr_dim))            
        word = pieces[0]            
        vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")            
        nlp.vocab.set_vector(word, vector)  # add the vectors to the vocab   

In [19]:
text = "भारी बारिश के कारण आज कार्यालय बंद रहेगा"    
doc = nlp(text)    
print("similarity btw",doc[0] , "and", doc[3]," :-", doc[0].similarity(doc[3]))

similarity btw भारी and कारण  :- 0.41513643


# Part  2

Steps involved :- 
1. Loading required library
2. Setting up the parameters for traning
3. Traning and saving the best model

In [0]:
import spacy
import random # random function for to remove bais in Traning Data

# for batch parsing 
from spacy.util import minibatch, compounding

# For evaluateing the model from testing set
from spacy.gold import GoldParse
from spacy.scorer import Scorer


In [0]:
def train_spacy(TRAIN_DATA,TEST_DATA,iterations,droprate = 0.5,modelName = "modelTrained"):

    # loading hindi model and using vector from fasttext
    lang = "hi"
    vectors_loc = "wiki.hi.vec"
    modiner = spacy.blank(lang)    
    with open(vectors_loc, "rb") as file_:        
        header = file_.readline()        
        nr_row, nr_dim = header.split()        
        modiner.vocab.reset_vectors(width=int(nr_dim))        
        for line in file_:            
            line = line.rstrip().decode("utf8")            
            pieces = line.rsplit(" ", int(nr_dim))            
            word = pieces[0]            
            vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")            
            modiner.vocab.set_vector(word, vector)  # add the vectors to the vocab    

#     modiner = spacy.blank('en')  # create blank Language class
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in modiner.pipe_names:
        ner = modiner.create_pipe('ner')
        modiner.add_pipe(ner, last=True)
     
    # setting up f1score
    f1score = 0.0000

    
    # add labels that will be involved in training 
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
            

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in modiner.pipe_names if pipe != 'ner']
    with modiner.disable_pipes(*other_pipes):  # only train NER
        optimizer = modiner.begin_training()
        
        # --Iterations Starts--
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            #--Shuffling Traning Data--
            random.shuffle(TRAIN_DATA)
            losses = {}
            
                      
                    
            # batch Traning For better Training and Learning of model
            batches = minibatch(TRAIN_DATA, size=compounding(2.0, 16.0, 1.01))
            for batch in batches:
                texts, annotations = zip(*batch)
                modiner.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=droprate,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print(losses)
            
            
            # Evaluating the Current Model Score on test data
            results = evaluate(modiner, TEST_DATA)
            print("Current Score :-",results["ents_f"], "Precision  :-",results["ents_p"], "Recall  :-",results["ents_r"])
            

            
            # loading previous best saved model in start of traning 
            if f1score == 0.00:
                try:                    
                    pnlp = spacy.load(modelName)
                    result = evaluate(pnlp, TEST_DATA) # calling evaluate function 
                    f1score = result["ents_f"]
                except:
                    print("Previous Model not found")
                    
            print("Best Sccore :- ",f1score)
            print("------------------------------------")
            # finding out the best score
            if f1score < results["ents_f"]:
                f1score = results["ents_f"]
                
                # Save our trained Model if the score if grater than best score else no change in previous model
                modiner.to_disk(modelName)
                
    print("-----Best Model is Saved-----")
     


In [0]:
def evaluate(ner_model, examples):
    scorer = Scorer()
     
    #loading tags for each input and Evaluating them
    for input_, annotations in examples:
        tags = []
        # loading text
        doc_gold_text = ner_model.make_doc(input_)
        
        #loading all tags for that text
        for ent in annotations.get('entities'):
            tags.append(ent)
            
        # Evaluating the tags    
        gold = GoldParse(doc_gold_text, entities=tags)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
        
        
    return scorer.scores


In [0]:
def loadNERModel(modelName = "modelTrained"):
    nlp = spacy.load(modelName)
    return nlp

In [0]:
def score(model,TEST_DATA):
    result = evaluate(model, TEST_DATA) # calling evaluate function 
    f1score = result["ents_f"]
    precision = result["ents_p"]
    recall = result["ents_r"]
    print("F1 score of Model is :-",f1score)
    print("Precision of Model is :-",precision)
    print("Recall of Model is :-",recall)

In [25]:
# traning the model with 100 iterations


train_spacy(TRAIN_DATA,TEST_DATA, 100,droprate = 0.55, modelName = "hindiNER")

Starting iteration 0
{'ner': 790.6779609294608}
Current Score :- 23.52941176470588 Precision  :- 100.0 Recall  :- 13.333333333333334
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 1
{'ner': 485.5736994091858}
Current Score :- 81.81818181818184 Precision  :- 83.72093023255815 Recall  :- 80.0
Best Sccore :-  23.52941176470588
------------------------------------
Starting iteration 2
{'ner': 330.1836383179055}
Current Score :- 86.66666666666667 Precision  :- 86.66666666666667 Recall  :- 86.66666666666667
Best Sccore :-  81.81818181818184
------------------------------------
Starting iteration 3
{'ner': 256.2229168736599}
Current Score :- 93.47826086956522 Precision  :- 91.48936170212765 Recall  :- 95.55555555555556
Best Sccore :-  86.66666666666667
------------------------------------
Starting iteration 4
{'ner': 184.24577634265853}
Current Score :- 97.82608695652173 Precision  :- 95.74468085106383 Recall  :- 100.0
Best Sccore :-  93.4

In [0]:
# loading the saved model
pnlp = loadNERModel("hindiNER")

In [27]:
# calculating the score of the model
score(pnlp,TEST_DATA)

F1 score of Model is :- 97.82608695652173
Precision of Model is :- 95.74468085106383
Recall of Model is :- 100.0


In [0]:
from spacy import displacy

In [29]:
testcase = pnlp(TEST_DATA[0][0])
displacy.render(testcase, style='ent', jupyter=True)

# Training Data Result

In [30]:

for i in range(0,len(TRAIN_DATA)):
    testcase = pnlp(TRAIN_DATA[i][0])
    displacy.render(testcase, style='ent', jupyter=True)
    print("--")

--


--


--


--


--


...

--


--


--


--


--


# Testing Data Result

In [31]:
for i in range(0,len(TEST_DATA)):
    try:        
        testcase = pnlp(TEST_DATA[i][0])
        displacy.render(testcase, style='ent', jupyter=True)
        print("--")
    except:
        a=None

--


--


--


--


--


...

--


--


--


--


--


# Part 3 :-
## training with 50 and testing 170

In [32]:

data = TRAIN_DATA + TEST_DATA

len(data)

224

In [0]:
TRAIN_DATA = data[:50]
TEST_DATA = data[50:]

In [34]:
# traning the model with 20 iterations


train_spacy(TRAIN_DATA,TEST_DATA, 100,droprate = 0.55, modelName = "hindiNER1")

Starting iteration 0
{'ner': 262.327996712178}
Current Score :- 0.0 Precision  :- 0.0 Recall  :- 0.0
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 1
{'ner': 194.39351097680628}
Current Score :- 0.0 Precision  :- 0.0 Recall  :- 0.0
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 2
{'ner': 185.1396162956953}
Current Score :- 0.5249343832020998 Precision  :- 100.0 Recall  :- 0.2631578947368421
Previous Model not found
Best Sccore :-  0.0
------------------------------------
Starting iteration 3
{'ner': 154.98514048825018}
Current Score :- 58.805513016845346 Precision  :- 70.32967032967034 Recall  :- 50.526315789473685
Best Sccore :-  0.5249343832020998
------------------------------------
Starting iteration 4
{'ner': 126.75471179503074}
Current Score :- 67.85714285714286 Precision  :- 70.97701149425288 Recall  :- 65.0
Best Sccore :-  58.805513016845346
------------------------------

In [35]:
# loading the saved model
pnlp = loadNERModel("hindiNER")
# calculating the score of the model
score(pnlp,TEST_DATA)

F1 score of Model is :- 91.05058365758755
Precision of Model is :- 89.769820971867
Recall of Model is :- 92.36842105263158


In [0]:
# !zip -r /content/hindiNER.zip /content/hindiNER

In [0]:
# !rm -rf  hindiNER